# Check that hemispheres are:
# 1. not identical
# 2. correclty assigned


In [ ]:
import numpy as np
import pandas as pd
import os
os.chdir("/host/verges/tank/data/daniel/01_3T7T/z/code/analyses/")
import tTsTGrpUtils as tsutil
import utils_plots as up
import importlib
import datetime 

In [ ]:
dl_pth = "/host/verges/tank/data/daniel/3T7T/z/outputs/04d_dl_maps_26Nov2025-153407.pkl"
dl = tsutil.loadPickle(dl_pth)

log_pth = f"/host/verges/tank/data/daniel/01_3T7T/z/outputs/debug/logs/mapDifs_{datetime.datetime.now().strftime('%d%b%Y-%H%M%S')}"
logger = tsutil._get_file_logger(__name__, log_file_path=log_pth)
print(f"Logging to: {log_pth}")
tsutil.print_dict(dl)

for i, itm in enumerate(dl):
    print(f"{i}/{len(dl)}...")
    logger.info(f"{'-'*100}\n{tsutil.printItemMetadata(itm, return_txt=True)}") 
    mps = tsutil.loadPickle(itm['df_maps'])
    mps_L, mps_R = tsutil.splitHemis(mps, rmv_lbl=True)
    mps_d = mps_L - mps_R
    # print summary stats
    logger.info("\tDifference L-R summary stats:")
    
    # Summary stats across all vertices
    all_stats = pd.Series({
        'name': 'all',
        'count': mps_d_values.size,
        'mean': np.nanmean(mps_d_values),
        'std': np.nanstd(mps_d_values),
        'min': np.nanmin(mps_d_values),
        '25%': np.nanpercentile(mps_d_values, 25),
        '50%': np.nanpercentile(mps_d_values, 50),
        '75%': np.nanpercentile(mps_d_values, 75),
        'max': np.nanmax(mps_d_values)
    })

    # Fast binning using numpy for better performance
    n_bins = 20
    # Convert to numpy for faster computation
    mps_d_values = mps_d.values
    
    # Calculate bin edges
    bin_edges = np.linspace(0, mps_d_values.shape[0], n_bins + 1, dtype=int)
    
    binned_stats = []

    binned_stats.append(all_stats)

    for i in range(n_bins):
        start_idx = bin_edges[i]
        end_idx = bin_edges[i + 1]
        bin_data = mps_d_values[start_idx:end_idx]
        
        # Calculate stats directly with numpy (much faster)
        bin_stats = pd.Series({
            'name': f'bin_{i+1}',
            'count': bin_data.size,
            'mean': np.nanmean(bin_data),
            'std': np.nanstd(bin_data),
            'min': np.nanmin(bin_data),
            '25%': np.nanpercentile(bin_data, 25),
            '50%': np.nanpercentile(bin_data, 50),
            '75%': np.nanpercentile(bin_data, 75),
            'max': np.nanmax(bin_data)
        })
        binned_stats.append(bin_stats)
    

    # Combine all bin statistics into a single DataFrame
    binned_df = pd.concat(binned_stats, axis=1)
    logger.info(binned_df.round(4).to_string())

    # create plot. plot mean and std across bins. colour cortical and hippocampal surfaces differently
    color = 'C0' if 'cortical' in itm['map_name'].lower() else 'C1'
    

In [ ]:
# print name of functions in tsutil
funcs = [f for f in dir(tsutil) if callable(getattr(tsutil, f)) and not f.startswith("_")]
print("Functions in tTsTGrpUtils:")
for f in funcs:
    print(f)

In [ ]:
importlib.reload(tsutil)


In [ ]:
# debug using z-brains outputted smoothed maps
input_df = '/host/verges/tank/data/daniel/01_3T7T/z/outputs/04c_dfPths_dsMaps_26Nov2025-132106.csv'

tsutil.extractMap()